# Setup e carregamentos

In [1]:
# =========================
# Setup e configurações
# =========================
import os
os.chdir(r'C:\Projetos\case-ami-saude')

import pandas as pd
import numpy as np
from src.utils import pipeline_universal_limpeza, validacao_cid, trat_senha_internacao

In [2]:
# =========================
# Carregamento da Base 01
# =========================
path_base_01 = "C:\\Projetos\\case-ami-saude\\data\\raw\\Base 1 – Internações Hospitalares.csv"
df_base_01 = pd.read_csv(path_base_01)

df_base_01.shape


(20000, 36)

In [3]:
# =========================
# Carregamento da Base 02
# =========================
path_base_02 = "C:\\Projetos\\case-ami-saude\\data\\raw\\Base 2 – Itens da internação.csv"
df_base_02 = pd.read_csv(path_base_02)

df_base_02.shape


(400000, 17)

In [4]:
# Tratamentos dos tipos de dados
df_base_01 = pipeline_universal_limpeza(df_base_01)
df_base_02 = pipeline_universal_limpeza(df_base_02)

In [5]:
# Coluna de status da internação

df_base_01["status_internacao"] = "ENCERRADA"
df_base_01.loc[df_base_01["data_alta"].isna(), "status_internacao"] = "ATIVA"


In [6]:
df_base_01

,senha_internacao,beneficiario_id,numero_carteirinha,nome_beneficiario,data_nascimento,idade,sexo,uf,municipio,hospital_id,...,suporte_ventilatorio_flag,hemodialise_flag,tempo_autorizacao_horas,auditoria_responsavel,empresa_auditoria,status_regulacao,glosa_flag,valor_total_conta,valor_pago,status_internacao
0,SI20250000001,SI20250000001,CAR405957,P.M.M.,1992-06-17,33,M,SP,Santos,H070,...,Não,Não,30.0,Terceirizada,Empresa Terceira 3,Autorizada,Não,112005.19,112005.19,ENCERRADA
1,SI20250000002,SI20250000002,CAR658953,V.S.M.,1987-10-24,38,F,MS,Dourados,H013,...,Não,Sim,22.0,Terceirizada,Empresa Terceira 3,Autorizada com ressalva,Sim,357271.95,303110.50,ENCERRADA
2,SI20250000003,SI20250000003,CAR547463,N.M.A.,1929-02-16,96,F,MA,Imperatriz,H001,...,Não,Não,26.0,Terceirizada,Empresa Terceira 2,Pendente,Não,32603.64,32603.64,ENCERRADA
3,SI20250000004,SI20250000004,CAR181581,P.C.N.,1976-10-29,49,M,MG,Juiz de Fora,H043,...,Não,Não,34.0,Terceirizada,Empresa Terceira 2,Autorizada,Sim,24660.48,23423.61,ENCERRADA
4,SI20250000005,SI20250000005,CAR677468,B.A.R.,1981-02-07,45,M,PA,Belém,H028,...,Não,Sim,21.0,Terceirizada,Empresa Terceira 3,Autorizada,Sim,46771.29,40277.38,ENCERRADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,SI20250019996,SI20250019996,CAR811091,P.N.C.,1983-05-08,42,F,SP,Santos,H043,...,Não,Não,29.0,Terceirizada,Empresa Terceira 2,Pendente,Não,18658.17,18658.17,ENCERRADA
19996,SI20250019997,SI20250019997,CAR148487,L.M.T.,1952-11-05,73,M,MA,Imperatriz,H008,...,Sim,Não,27.0,Terceirizada,Empresa Terceira 1,Autorizada,Não,30272.52,30272.52,ENCERRADA
19997,SI20250019998,SI20250019998,CAR540753,M.F.M.,1995-06-28,30,F,PR,Curitiba,H061,...,Não,Não,13.0,Terceirizada,Empresa Terceira 2,Autorizada,Não,22906.91,22906.91,ENCERRADA
19998,SI20250019999,SI20250019999,CAR367435,A.M.O.,1948-11-02,77,F,MG,Contagem,H007,...,Não,Não,40.0,Terceirizada,Empresa Terceira 2,Autorizada,Não,12890.38,12890.38,ENCERRADA


In [7]:
# Coluna tempo de permanencia dos pacientes

df_base_01["dias_internado"] = (
    df_base_01["data_alta"]
    .fillna(pd.Timestamp.today())
    - df_base_01["data_admissao"]
).dt.days



In [8]:
# Coluna idade:
adm  = pd.to_datetime(df_base_01["data_admissao"], errors="coerce")
nasc = pd.to_datetime(df_base_01["data_nascimento"], errors="coerce")

idade_aprox = (adm - nasc).dt.days / 365.25

# forçar numérico (se tiver lixo, vira NaN)
idade_aprox = pd.to_numeric(idade_aprox, errors="coerce")

# se quiser idade inteira "aprox" truncada (tipo floor)
df_base_01["idade"] = np.floor(idade_aprox).astype("Int64")

In [9]:
# Correção em tempo de autorização
delta = (
    df_base_01['data_autorizacao_senha'] -
    df_base_01['data_solicitacao_autorizacao']
)
df_base_01['tempo_autorizacao_horas'] = delta.dt.total_seconds() / 3600

In [10]:
df_base_01 = validacao_cid(df_base_01)

In [11]:
df_base_01

,senha_internacao,beneficiario_id,numero_carteirinha,nome_beneficiario,data_nascimento,idade,sexo,uf,municipio,hospital_id,...,cid_secundario_compativel_especialidade,alerta_swap_obstetrica,alerta_swap_psiquiatrica,cidp_comp_esp,cids_comp_esp,alerta_swap_especialidade,alerta_swap_generico,alerta_swap_carater,alerta_swap,score_swap
0,SI20250000001,SI20250000001,CAR405957,P.M.M.,1992-06-17,33,M,SP,Santos,H070,...,True,False,False,False,True,True,False,False,True,1
1,SI20250000002,SI20250000002,CAR658953,V.S.M.,1987-10-24,38,F,MS,Dourados,H013,...,True,False,False,False,True,True,False,False,True,1
2,SI20250000003,SI20250000003,CAR547463,N.M.A.,1929-02-16,96,F,MA,Imperatriz,H001,...,True,False,False,False,True,True,False,False,True,1
3,SI20250000004,SI20250000004,CAR181581,P.C.N.,1976-10-29,48,M,MG,Juiz de Fora,H043,...,True,False,False,False,True,True,False,False,True,1
4,SI20250000005,SI20250000005,CAR677468,B.A.R.,1981-02-07,43,M,PA,Belém,H028,...,True,False,False,False,True,True,False,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,SI20250019996,SI20250019996,CAR811091,P.N.C.,1983-05-08,41,F,SP,Santos,H043,...,True,False,False,True,True,False,False,False,False,0
19996,SI20250019997,SI20250019997,CAR148487,L.M.T.,1952-11-05,71,M,MA,Imperatriz,H008,...,False,False,False,True,False,False,False,False,False,0
19997,SI20250019998,SI20250019998,CAR540753,M.F.M.,1995-06-28,30,F,PR,Curitiba,H061,...,False,False,False,False,False,False,False,False,False,0
19998,SI20250019999,SI20250019999,CAR367435,A.M.O.,1948-11-02,76,F,MG,Contagem,H007,...,True,False,False,False,True,True,False,False,True,1


In [12]:
df_base_01 = trat_senha_internacao(df_base_01)
df_base_02 = trat_senha_internacao(df_base_02)

In [14]:
df_base_01['senha_internacao_valida'].value_counts(dropna=False)

senha_internacao_valida
True    20000
Name: count, dtype: int64

In [27]:
df_base_02

,item_id,senha_internacao,data_item,tipo_item,subtipo_item,codigo_item,descricao_item,quantidade_solicitada,quantidade_autorizada,unidade_medida,valor_unitario,valor_total_item,setor_execucao,flag_pacote,glosa_item_flag,motivo_glosa,valor_glosado,senha_internacao_valida
0,IT000047081,SI20250002374,2025-04-23 16:36:36,Exame,Exames de imagem (un),EX_IMG,Exames de imagem (un),2,2,un,1675.92,3351.84,SADT,Não,Sim,Administrativa - código incompatível,3351.84,True
1,IT000290746,SI20250014538,2024-08-17 05:07:19,Taxa,Honorários (un),TX_HON,Honorários (un),3,3,un,1158.60,3475.80,Administrativo,Não,Sim,Técnica - item não pertinente ao CID,3475.80,True
2,IT000341194,SI20250017099,2024-10-12 09:09:58,Procedimento,Procedimento Cirúrgico,PR_CIR,Procedimento Cirúrgico,1,1,un,4314.57,4314.57,Centro Cirúrgico,Não,Não,NaN,0.00,True
3,IT000179585,SI20250008978,2025-10-23 17:20:13,Diária,Diária Enfermaria,DI_ENF,Diária Enfermaria,8,8,diária,559.87,4478.96,Enfermaria,Não,Não,NaN,0.00,True
4,IT000367705,SI20250018405,2024-07-31 13:28:24,Medicamento,Medicamentos (dose),MD_GER,Medicamentos (dose),4,4,dose,89.72,358.88,Farmácia,Não,Não,NaN,0.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,IT000401849,SI20250011218,2025-06-23 13:45:13,Diária,Diária Enfermaria,DI_ENF,Diária Enfermaria,17,17,diária,1369.52,23281.77,Enfermaria,Não,Não,NaN,0.00,True
399996,IT000401850,SI20250004187,2025-10-11 17:31:34,Medicamento,Antibiótico (dose),MD_ATB,Antibiótico (dose),2,2,dose,62.85,125.70,Farmácia,Não,Não,NaN,0.00,True
399997,IT000401851,SI20250011297,2025-06-20 20:24:33,Taxa,Taxas e materiais (un),TX_MAT,Taxas e materiais (un),1,0,un,210.93,0.00,Enfermaria,Não,Não,NaN,0.00,True
399998,IT000401852,SI20250009120,2025-11-21 00:11:00,Terapia seriada,Fonoaudiologia (sessão),TS_FON,Fonoaudiologia (sessão),4,4,sessão,84.44,337.76,Enfermaria,Não,Não,NaN,0.00,True


In [31]:
# Flags úteis
df_base_02["item_glosado_flag"] = df_base_02["valor_glosado"].fillna(0) > 0
df_base_02["delta_qtd"] = df_base_02["quantidade_solicitada"] - df_base_02["quantidade_autorizada"]
df_base_02["delta_qtd_flag"] = df_base_02["delta_qtd"].fillna(0) > 0

# Função de percentil (pandas não tem p95 direto no agg sem lambda)
def p95(s: pd.Series) -> float:
    s = s.dropna()
    return float(np.percentile(s, 95)) if len(s) else np.nan

agg_itens = (
    df_base_02
    .groupby("senha_internacao", as_index=False)
    .agg(
        itens_qtd_total=("item_id", "count"),
        itens_tipos_distintos=("tipo_item", "nunique"),
        itens_subtipos_distintos=("subtipo_item", "nunique"),
        itens_codigos_distintos=("codigo_item", "nunique"),

        itens_valor_total=("valor_total_item", "sum"),
        itens_valor_medio=("valor_total_item", "mean"),
        itens_valor_max=("valor_total_item", "max"),
        itens_valor_p95=("valor_total_item", p95),

        glosa_item_qtd=("item_glosado_flag", "sum"),
        valor_glosado_total=("valor_glosado", "sum"),
        motivos_glosa_distintos=("motivo_glosa", "nunique"),

        delta_qtd_total=("delta_qtd", "sum"),
        delta_qtd_itens_qtd=("delta_qtd_flag", "sum"),
    )
)

# Pós-agg: percentuais (evita divisão por zero)
agg_itens["glosa_item_pct"] = agg_itens["glosa_item_qtd"] / agg_itens["itens_qtd_total"].replace(0, np.nan)
agg_itens["valor_glosado_pct"] = agg_itens["valor_glosado_total"] / agg_itens["itens_valor_total"].replace(0, np.nan)
agg_itens["delta_qtd_pct_itens"] = agg_itens["delta_qtd_itens_qtd"] / agg_itens["itens_qtd_total"].replace(0, np.nan)

agg_itens

,senha_internacao,itens_qtd_total,itens_tipos_distintos,itens_subtipos_distintos,itens_codigos_distintos,itens_valor_total,itens_valor_medio,itens_valor_max,itens_valor_p95,glosa_item_qtd,valor_glosado_total,motivos_glosa_distintos,delta_qtd_total,delta_qtd_itens_qtd,glosa_item_pct,valor_glosado_pct,delta_qtd_pct_itens
0,SI20250000001,20,5,9,9,105888.66,5294.433000,17707.91,15334.335,0,0.00,0,22,2,0.000000,0.000000,0.100000
1,SI20250000002,27,8,12,12,303439.62,11238.504444,69747.67,27892.387,4,54161.45,4,19,4,0.148148,0.178492,0.148148
2,SI20250000003,13,6,8,8,35417.69,2724.437692,6458.04,5228.592,0,0.00,0,0,0,0.000000,0.000000,0.000000
3,SI20250000004,18,5,8,8,24660.48,1370.026667,7953.76,6339.882,2,1236.87,2,0,0,0.111111,0.050156,0.000000
4,SI20250000005,21,7,11,11,39977.75,1903.702381,5531.50,4655.810,4,6493.91,3,15,4,0.190476,0.162438,0.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000,SX52954311,1,1,1,1,450.00,450.000000,450.00,450.000,0,0.00,0,0,0,0.000000,0.000000,0.000000
20001,SX54154075,1,1,1,1,450.00,450.000000,450.00,450.000,0,0.00,0,0,0,0.000000,0.000000,0.000000
20002,SX67887514,1,1,1,1,450.00,450.000000,450.00,450.000,0,0.00,0,0,0,0.000000,0.000000,0.000000
20003,SX69652955,1,1,1,1,450.00,450.000000,450.00,450.000,0,0.00,0,0,0,0.000000,0.000000,0.000000


In [32]:
df_ml = df_base_01.merge(agg_itens, on="senha_internacao", how="left")